In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
pip install lancedb

In [ ]:
import lancedb

In [ ]:
!pip install open_clip_torch

In [ ]:
import open_clip

In [ ]:
model, preprocess = open_clip.create_model_from_pretrained('hf-hub:laion/CLIP-ViT-g-14-laion2B-s12B-b42K')
tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-g-14-laion2B-s12B-b42K')

In [ ]:
from lancedb.embeddings import EmbeddingFunctionRegistry

registry = EmbeddingFunctionRegistry.get_instance()
clip = registry.get("open-clip").create()

In [ ]:
clip

In [ ]:
from PIL import Image
from lancedb.pydantic import LanceModel, Vector

class Pets(LanceModel):
    vector: Vector(clip.ndims()) = clip.VectorField()
    image_uri: str = clip.SourceField()

    @property
    def image(self):
        return Image.open(self.image_uri)

In [ ]:
db = lancedb.connect("/lancedb")

In [ ]:
db.drop_table("pets")

In [ ]:
import pandas as pd
from pathlib import Path
from random import sample

if "pets" in db:
    table = db["pets"]
else:
    table = db.create_table("pets", schema=Pets)
    local_folder = "C:/Users/f.zarei/VDB/images1/images"  
    p = Path(local_folder).expanduser()
    # p = Path("/content/gdrive/MyDrive/images/images").expanduser()
    uris = [str(f) for f in p.glob("*.jpg")]
    uris = sample(uris, 1000)
    table.add(pd.DataFrame({"image_uri": uris}))

In [ ]:

table_list = db.list_tables()
print(f"Tables in database: {table_list}")

In [ ]:
table.head().to_pandas()

In [ ]:
rs = table_pets.search("cat").limit(3).to_pydantic(Pets)
rs[2].image

In [ ]:
rs = table_pets.search("cat").limit(3).to_pydantic(Pets)
print(f"Number of results: {len(rs)}")

In [ ]:
from PIL import Image
p = Path("/content/gdrive/MyDrive/images/images/British_Shorthair_241.jpg").expanduser()
query_image = Image.open(p)
query_image

In [ ]:
rs = table.search(query_image).limit(3).to_pydantic(Pets)
rs[1].image